### This notebook was designed to make changes to number of neurons and activation codes easier

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_cts = application_df.APPLICATION_TYPE.value_counts()
application_cts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_cts[application_cts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_cts = application_df.CLASSIFICATION.value_counts()
class_cts.head(50)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1246        2
C1234        2
C3200        2
C1267        2
C1256        2
C4200        1
C2150        1
C2600        1
C1283        1
C4120        1
Name: CLASSIFICATION, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_cts[class_cts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
# NOTE CREATING NEW DATAFRAME
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
application_df.columns.tolist()

['STATUS',
 'ASK_AMT',
 'IS_SUCCESSFUL',
 'APPLICATION_TYPE_Other',
 'APPLICATION_TYPE_T10',
 'APPLICATION_TYPE_T19',
 'APPLICATION_TYPE_T3',
 'APPLICATION_TYPE_T4',
 'APPLICATION_TYPE_T5',
 'APPLICATION_TYPE_T6',
 'APPLICATION_TYPE_T7',
 'APPLICATION_TYPE_T8',
 'AFFILIATION_CompanySponsored',
 'AFFILIATION_Family/Parent',
 'AFFILIATION_Independent',
 'AFFILIATION_National',
 'AFFILIATION_Other',
 'AFFILIATION_Regional',
 'CLASSIFICATION_C1000',
 'CLASSIFICATION_C1200',
 'CLASSIFICATION_C2000',
 'CLASSIFICATION_C2100',
 'CLASSIFICATION_C3000',
 'CLASSIFICATION_Other',
 'USE_CASE_CommunityServ',
 'USE_CASE_Heathcare',
 'USE_CASE_Other',
 'USE_CASE_Preservation',
 'USE_CASE_ProductDev',
 'ORGANIZATION_Association',
 'ORGANIZATION_Co-operative',
 'ORGANIZATION_Corporation',
 'ORGANIZATION_Trust',
 'INCOME_AMT_0',
 'INCOME_AMT_1-9999',
 'INCOME_AMT_10000-24999',
 'INCOME_AMT_100000-499999',
 'INCOME_AMT_10M-50M',
 'INCOME_AMT_1M-5M',
 'INCOME_AMT_25000-99999',
 'INCOME_AMT_50M+',
 'INCOME_

In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Design neural network model

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import csv

# Model 1 - User input Deep Neural Net Model - 2 layer

In [70]:
# User Inputs

hn1 = 100
hn2 = 50
act1 = "tanh"
act2 = "relu"
act3 = "sigmoid"

In [71]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  hn1
hidden_nodes_layer2 = hn2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=act1))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=act2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation=act3))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 100)               4400      
_________________________________________________________________
dense_30 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 51        
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [72]:
# # Import checkpoint dependencies
# import os
# from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 860us/step - loss: 0.5669 - accuracy: 0.7225

Epoch 00001: saving model to checkpoints\weights.01.hdf5
Epoch 2/100
804/804 [==============================] - 1s 919us/step - loss: 0.5562 - accuracy: 0.7297

Epoch 00002: saving model to checkpoints\weights.02.hdf5
Epoch 3/100
804/804 [==============================] - 1s 921us/step - loss: 0.5540 - accuracy: 0.7305

Epoch 00003: saving model to checkpoints\weights.03.hdf5
Epoch 4/100
804/804 [==============================] - 1s 934us/step - loss: 0.5517 - accuracy: 0.7310

Epoch 00004: saving model to checkpoints\weights.04.hdf5
Epoch 5/100
804/804 [==============================] - 1s 922us/step - loss: 0.5506 - accuracy: 0.7335

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 1s 921us/step - loss: 0.5498 - accuracy: 0.7320

Epoch 00006: saving model to checkpoints\weights.06.hdf5
Epoch 7/100
804/804 [=============

In [73]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5721 - accuracy: 0.7248
Loss: 0.5721322298049927, Accuracy: 0.724781334400177


In [ ]:
# print(f"{hn1}, {hn2}, {act1}, {act2}, {act3}, {model_loss}, {model_accuracy}")
# history_df = pd.DataFrame.(f"{hn1}, {hn2}, {act1}, {act2}, {act3}, {model_loss}, {model_accuracy}")
# history

In [74]:
history = (hn1, hn2, act1, act2, act3, model_loss, model_accuracy)
history

(100, 50, 'tanh', 'relu', 'sigmoid', 0.5721322298049927, 0.724781334400177)

In [ ]:
# # create history_data.csv
# # use only ONCE to create csv. will overwrite previous results.
# header = ['hn1', 'hn2', 'act1', 'act2', 'act3', 'model_loss', 'model_accuracy', 'epochs']

# with open("./history_data.csv", 'w', newline='') as f:
#     # create the csv writer
#     writer = csv.writer(f)
    
#     # write the header
#     writer.writerow(header)

In [75]:
# Add row to history_data.csv
with open("./history_data.csv", 'a', newline='') as f:
    # create the csv writer
    writer = csv.writer(f)
    
    # write a row to the csv file
    writer.writerow(history)  

# Model 2 - User input Deep Neural Net Model - 3 layer¶

In [64]:
# User Inputs

hn1 = 100
hn2 = 50
hn3 = 50
act1 = "tanh"
act2 = "relu"
act3 = "relu"
act4 = "relu"

In [65]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  hn1
hidden_nodes_layer2 = hn2
hidden_nodes_layer3 = hn3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=act1))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=act2))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=act3))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation=act4))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 100)               4400      
_________________________________________________________________
dense_26 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_27 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 51        
Total params: 12,051
Trainable params: 12,051
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 980us/step - loss: 1.2703 - accuracy: 0.6879

Epoch 00001: saving model to checkpoints\weights.01.hdf5
Epoch 2/100
804/804 [==============================] - 1s 989us/step - loss: 0.5880 - accuracy: 0.7239

Epoch 00002: saving model to checkpoints\weights.02.hdf5
Epoch 3/100
804/804 [==============================] - 1s 996us/step - loss: 0.7683 - accuracy: 0.7056

Epoch 00003: saving model to checkpoints\weights.03.hdf5
Epoch 4/100
804/804 [==============================] - 1s 992us/step - loss: 0.5712 - accuracy: 0.7288

Epoch 00004: saving model to checkpoints\weights.04.hdf5
Epoch 5/100
804/804 [==============================] - 1s 987us/step - loss: 0.5767 - accuracy: 0.7245

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 1s 996us/step - loss: 0.5886 - accuracy: 0.7251

Epoch 00006: saving model to checkpoints\weights.06.hdf5
Epoch 7/100
804/804 [=============

804/804 [==============================] - 1s 1ms/step - loss: 0.5718 - accuracy: 0.7347

Epoch 00052: saving model to checkpoints\weights.52.hdf5
Epoch 53/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5608 - accuracy: 0.7346

Epoch 00053: saving model to checkpoints\weights.53.hdf5
Epoch 54/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5699 - accuracy: 0.7313

Epoch 00054: saving model to checkpoints\weights.54.hdf5
Epoch 55/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5660 - accuracy: 0.7355

Epoch 00055: saving model to checkpoints\weights.55.hdf5
Epoch 56/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5698 - accuracy: 0.7339

Epoch 00056: saving model to checkpoints\weights.56.hdf5
Epoch 57/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7374

Epoch 00057: saving model to checkpoints\weights.57.hdf5
Epoch 58/100
804/804 [==============================]

In [67]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6123 - accuracy: 0.7233
Loss: 0.6122683882713318, Accuracy: 0.7232652902603149


In [68]:
history2 = (hn1, hn2, hn3, act1, act2, act3, act4, model_loss, model_accuracy)
history2

(100,
 50,
 50,
 'tanh',
 'relu',
 'relu',
 'relu',
 0.6122683882713318,
 0.7232652902603149)

In [39]:
# # create history_data2.csv
# # use only ONCE to create csv. will overwrite previous results.
# header = ['hn1', 'hn2', 'hn3', 'act1', 'act2', 'act3', 'act4', 'model_loss', 'model_accuracy', 'epochs']

# with open("./history_data.2.csv", 'w', newline='') as f:
#     # create the csv writer
#     writer = csv.writer(f)
    
#     # write the header
#     writer.writerow(header)


In [69]:
# Add row to history_data2.csv
with open("./history_data.2.csv", 'a', newline='') as f:
    # create the csv writer
    writer = csv.writer(f)
    
    # write a row to the csv file
    writer.writerow(history2)  